<a href="https://colab.research.google.com/github/papagorgio23/Python101/blob/master/Optimizing_a_Library_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Stocking a digital library using combinatorial optimization

## Background
Suppose we are starting a library and want to select the best set of books. 

There are so many great books available, but we only have $1000 to spend. Which books do we select?

## Solution
This is an example of the Knapsack problem. We have a knapsack of limited capacity and items of varying sizes and utility; which items do we pack?

In this scenario, the "knapsack" is the $1000 book budget and the items are the books. Each book has an associated "enjoyment" level and an associated cost.

Using combinatorial optimization, we will find the optimal set of books to maximize reader enjoyment and stay within the budget.

The following code is adapted from Google's [OR tools knapsack](https://developers.google.com/optimization/bin/knapsack) example.

In [ ]:
# install Google's OR tools library
%pip install --upgrade --user ortools

In [ ]:
from ortools.algorithms import pywrapknapsack_solver
import pandas as pd
import math
import time

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/'

### 1. Load the data

The [Goodreads dataset](https://www.kaggle.com/jealousleopard/goodreadsbooks) will serve as the universe of books to choose from. 

For illustrative purposes, we'll assume the cost of each book is proportional to the number pages, at $0.10/page.

We'll also assume that "best" is proportional to the average user rating. Therefore, we want to select the set of books that has the maximal overall user rating.

In [ ]:
books = pd.read_csv("books.csv")

In [ ]:
books["price"] = books['  num_pages']*0.10

In [ ]:
books.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,price
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,65.2
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,87.0
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,35.2
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,43.5
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,269.0


### 2. Instantiate the solver

More info on the knapsack solver can be found [here](https://developers.google.com/optimization/reference/python/algorithms/pywrapknapsack_solver).


We also have to convert the ratings and prices to integers since the solver only works for integer values.

Since the ratings range from 0 to 5.00, we'll multiply by 100. For the price, we'll round to the nearest dollar.

In [ ]:
solver = pywrapknapsack_solver.KnapsackSolver(
    pywrapknapsack_solver.KnapsackSolver.
    KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample')

In [ ]:
# set the limit of the "knapsack"

MAX_BUDGET = [1000]

# declare the value and cost of each book and convert to integers
ratings = list(map(lambda x: int(x*100), books.average_rating.to_list()))
prices = [list(map(lambda x: round(x), books.price.to_list()))]

### 3. Run the solver

Call the solver with the ratings and price of each book.

In [ ]:
solver.Init(ratings, prices, MAX_BUDGET)
computed_value = solver.Solve()

### 4. Display the best solution

The solver's `BestSolutionContains` method holds an array indicating which items are in the optimal set. Let's see which books it selected.



In [ ]:
selected_books = []
total_cost = 0
avg_rating = 0
for i in range(len(ratings)):
    if solver.BestSolutionContains(i):
        selected_books.append((books.iloc[i].title,ratings[i]))
        total_cost += prices[0][i]
        avg_rating += ratings[i]
        # print(i,books.iloc[i].title,ratings[i])
print('The total cost of the libary is:', total_cost)
print("The library has %d books" % len(selected_books))
print('Library average rating: %0.1f' % (avg_rating/(100*len(selected_books))))

print('\n')
print('Selected books:')
sorted_books = sorted(selected_books,key=lambda x: x[1],reverse=True)
for book in sorted_books:
   print("%s - Rating: %0.1f" % (book[0], book[1]/100.0))


The total cost of the libary is: 1000
The library has 526 books
Library average rating: 4.0


Selected books:
Literature Circle Guide: Bridge to Terabithia: Everything You Need For Successful Literature Circles That Get Kids Thinking  Talking  Writing—and Loving Literature - Rating: 5.0
The Goon Show  Volume 4: My Knees Have Fallen Off! - Rating: 5.0
The Goon Show  Volume 11: He's Fallen in the Water! - Rating: 5.0
Tyrannosaurus Wrecks (Stanley  #1) - Rating: 5.0
Bill Gates: Computer Legend (Famous Lives) - Rating: 5.0
The Feynman Lectures on Physics Vols 7-8 - Rating: 4.8
The Feynman Lectures on Physics Vols 3-4 - Rating: 4.7
The 5 Love Languages / The 5 Love Languages Journal - Rating: 4.7
Bill Buzz - Rating: 4.7
Code Check Electrical: An Illustrated Guide to Wiring a Safe House - Rating: 4.7
Herbert the Timid Dragon - Rating: 4.6
The Feynman Lectures on Physics  3 Vols - Rating: 4.6
The Feynman Lectures on Physics Vols 5-6 - Rating: 4.6
Vinyl Cafe Odd Jobs - Rating: 4.5
Harry Potter

## Summary

We've demonstrated a simple example of how to use Google's OR tools to optimize a library, which didn't require a lot of work to set up. A further extension to this problem is to find an optimial library collections for many users. This is the *multi-knapsack problem* and is also available in the OR Tools package.